In [2]:
import pytest
import ipytest
import functools
import numpy as np
import copy
import warnings

ipytest.autoconfig()

In [3]:
keyfy = lambda lst: str(lst).strip('[]').replace(' ', '')
reduce = lambda func, elems, elem0: functools.reduce(func, elems+[elem0])
unique = lambda lst: list(np.unique(lst))

In [4]:
def euler(sets):
    '''
        @abstract returns each tuple [key, elems] of the Euler diagram
        systematic in a generator-wise fashion
        
        Rationale: 
           1. Begin with the available sets and their exclusive elements;
           2. Compute complementary elements to current key-set;
           3. In case complementary set-keys AND current set content are not empty, continue;
           Otherwise, go to next key-set;
           4. Find the euler diagram on complementary sets;
           5. Compute exclusive combination elements;
           6. In case there are exclusive elements to combination:
           6.a Yield exclusive combination elements;
           6.b Remove exclusive combination elements from current key-set;

        @param {Array} sets
        @return {Array} keys_elems
    '''
    set_keys_ = keyfy(list(sets.keys()))
    
    if not reduce(
              lambda a, b: a and b,
              [
                  len(unique(values)) == len(values) 
                   for values in sets.values()
              ], True):
            
        warnings.warn("Each array MUST NOT have duplicates")
        sets = {key: unique(values) for key, values in sets.items()}
    
    # Only a set
    if (len(sets.values()) == 1): 
        key = list(sets.keys())[0]
        value = list(sets.values())[0]
        yield (key, value)
    
    else:
        # There are no sets
        if (not isinstance(sets, (list, dict))): 
            raise Exception("Ill-conditioned input.")
        
        # Remove empty sets
        sets_clear = lambda sets_: list(filter(lambda key: len(sets_[key]) != 0, sets_.keys()))

        # Keys complementary to current available key-set 
        compl_sets_keys = []

        # Sets with non-empty elements
        set_keys = sets_clear(sets); 

        # Traverse the combination lattice 
        for set_key in set_keys:
            compl_sets_keys = list(set(set_keys) - set([set_key]))
            
            # There are still sets to analyze
            # Morgan Rule: ¬(A & B) = ¬A | ¬B
            if (len(compl_sets_keys) != 0 and len(sets[set_key]) != 0):
                for comb_str, celements in euler({compl_set_key: sets[compl_set_key] 
                                                    for compl_set_key in compl_sets_keys}):
                    # Exclusive combination elements
                    comb_excl = list(set(celements)-set(sets[set_key]))
                    
                    # Non-empty combination exclusivity case
                    if(len(comb_excl) != 0):
                        # 1. Exclusive group elements except current analysis set
                        yield (comb_str, comb_excl)

                        for ckey in comb_str.split(','):
                            sets[ckey] = list(set(sets[ckey])-set(comb_excl))
                    
                    comb_intersec = list(set(celements).intersection(set(sets[set_key])))
                    sets[set_key] = list(set(sets[set_key]) - set(comb_intersec))

                    if (len(comb_intersec) != 0):
                        # 2. Intersection of analysis element and exclusive group
                        comb_intersec_key = set_key+','+comb_str
                        yield (comb_intersec_key, comb_intersec)

                        # Remove intersection elements from current key-set and complementary sets 
                        for ckey in comb_str.split(','):
                            sets[ckey] = list(set(sets[ckey])-set(comb_intersec))

                        sets[set_key] = list(set(sets[set_key])-set(comb_intersec))
                
                set_keys = sets_clear(sets)

                # 3. Set-key exclusive elements
                if (len(sets[set_key]) != 0):
                    yield (str(set_key), sets[set_key])

def spreadEuler(sets): 
    return {key: items for key, items in euler(sets)}

In [8]:
%%ipytest

# define the tests
def test_keyfy():
    assert keyfy([1, 2, 3]) == '1,2,3'

def test_one_set_euler():
    assert reduce(lambda a, b: a+b, [1, 2], 0) == 3

def test_unique_elems():
    assert unique([1, 2, 3]) == [1,2,3]

def test_euler_iter_1_input():
    assert next(euler({'a': [1,2]})) == ('a', [1,2])

def test_euler_iter_2_input():
    eu_fun = euler({'a': [1,2], 'b': [2,3]})
    
    assert next(eu_fun) == ('b', [3])
    assert next(eu_fun) == ('a,b', [2])
    assert next(eu_fun) == ('a', [1])
    
def test_euler_iter_error():
    with pytest.raises(Exception):
        next(euler(1))

def test_euler_iter_warning():
    with pytest.raises(Exception):
        next(euler(1))

def test_euler_iter_warning_1item():
    with pytest.warns(UserWarning):
        next(euler({'a': [42, 42]}))
        
def test_euler_iter_warning_2items():
    with pytest.warns(UserWarning):
        next(euler({'a': [42, 42], 'b': [42, 42]}))

def test_spread_euler_1_set():
    assert spreadEuler(
        {
            'a': [1, 2, 3]
        }
    ) == {'a': [1, 2, 3]}
    
def test_spread_euler_2_sets():
    assert spreadEuler(
        {
            'a': [1, 2, 3], 
            'b': [2, 3, 4]
        }
    ) == {'b': [4], 'a,b': [2, 3], 'a': [1]}

def test_spread_euler_3_sets():
    assert spreadEuler(
        {
            'a': [1, 2, 3], 
            'b': [2, 3, 4], 
            'c': [3, 4, 5]
        }
    ) == {'a,b': [2], 'c,b': [4], 'a,c,b': [3], 'c': [5], 'a': [1]}

def test_spread_euler_3_sets():
    assert spreadEuler(
        {
            'a': [1, 2, 3], 
            'b': [2, 3, 4], 
            'c': [3, 4, 5]
        }
    ) == {'a,b': [2], 'c,b': [4], 'a,c,b': [3], 'c': [5], 'a': [1]}

def test_spread_euler_4_sets():
    assert spreadEuler(
        {
            'a': [1, 2, 3], 
            'b': [2, 3, 4], 
            'c': [3, 4, 5],
            'd': [3, 5, 6]
        }
    ) == {'a,b': [2], 'c,b': [4], 'a,d,c,b': [3], 'd,c': [5], 'd': [6], 'a': [1]}

    

.............                                                                                [100%]
13 passed in 0.02s


In [11]:
print(
    spreadEuler(
        {
            'a': [1, 2, 3]
        }
    )
)

print(
    spreadEuler(
        {
            'a': [1, 2, 3], 
            'b': [2, 3, 4]
        }
    )
)

print(
    spreadEuler(
        {
            'a': [1, 2, 3], 
            'b': [2, 3, 4], 
            'c': [3, 4, 5]
        }
    )
)

print(
    spreadEuler(
        {
            'a': [1, 2, 3], 
            'b': [2, 3, 4], 
            'c': [3, 4, 5],
            'd': [3, 5, 6]
        }
    )
)

{'a': [1, 2, 3]}
{'b': [4], 'a,b': [2, 3], 'a': [1]}
{'a,b': [2], 'c,b': [4], 'a,c,b': [3], 'c': [5], 'a': [1]}
{'a,b': [2], 'c,b': [4], 'a,d,c,b': [3], 'd,c': [5], 'd': [6], 'a': [1]}
